## 1. Load the libraries

In [41]:
# update or install libraries
!pip install gradio
!pip install --upgrade openai

In [42]:
import spacy
import openai
import gradio as gr
import pandas as pd
import re
import IPython.display
from google.colab import files

In [47]:
openai.api_key = #Load API key

In [44]:
uploaded = files.upload()

Saving data.csv to data (3).csv


#2. Prompt Engineering

In [48]:
# Entity extraction function
def extract_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    return entities

# Summarization function
def generate_text(prompt):
    response = openai.Completion.create(
        engine="text-davinci-002",  # GPT-3
        prompt=prompt,
        max_tokens=50,
        temperature=0.7
    )
    return response.choices[0].text.strip()

# Sentiment and entity extraction
def classify_text(text):
    prompt_summary = f"""It's nice that they support English, but the menus are a bit strange in English. I wish I could set the language within the app.
    Explain the above in one sentence:"""

    generated_summary = generate_text(prompt_summary)

    entities = extract_entities(text)

    # Extract rating from the DataFrame
    review_row = df[df['review'] == text]
    if not review_row.empty:
        rating = review_row.iloc[0]['rating']
    else:
        rating = "Rating not found"

    prompt_extraction = f"""Extract entities (e.g., names, locations, product) from the review text:

    Text: {text}

    Entities:"""

    generated_extraction = generate_text(prompt_extraction)

    prompt_classification = f"""Classify the review text into negative or positive.

    Text: {text}

    Sentiment:"""

    generated_classification = generate_text(prompt_classification)

    return generated_extraction, generated_classification, generated_summary, rating

# Text Cleaning function
def clean_text(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\n\n", " ", text)
    text = re.sub(r"\t", " ", text)
    text = text.strip(" ")
    text = re.sub(" +", " ", text).strip()  # get rid of multiple spaces and replace with a single
    return text


# Load data from a CSV file
df = pd.read_csv('data.csv')

df.columns = ["rating", "review"]

# Apply text cleaning to the review column
df['review'] = df['review'].apply(clean_text)

# 3. Build an App UI with Gradio

In [49]:
# Gradio interface
app = gr.Interface(
    fn=classify_text,
    inputs=gr.inputs.Textbox(label="Enter a review:"),
    outputs=[
        gr.outputs.Textbox(label="Extracted Entities:"),
        gr.outputs.Textbox(label="Sentiment:"),
        gr.outputs.Textbox(label="Summary:"),
        gr.outputs.Textbox(label="Rating:")
    ],
    live=True,
    title="An app to review feedback for a product"
)

# Launch Gradio App
app.launch(share=True)

<ipython-input-49-d495aa294105>:4: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(label="Enter a review:"),
<ipython-input-49-d495aa294105>:4: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(label="Enter a review:"),
<ipython-input-49-d495aa294105>:4: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(label="Enter a review:"),
<ipython-input-49-d495aa294105>:6: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  gr.outputs.Textbox(label="Extracted Entities:"),
<ipython-input-49-d495aa294105>:7: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your component

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://acebffc69651a2c3ec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
